In [ ]:
# Dependencies - General Stuff
import numpy as np
import importlib
import tempfile
from scipy.signal import find_peaks

# strfpy
from strfpy import preprocSound, strfSetup, trnDirectFit, calcSegmentedModel
from strfpy.timeFreq import timefreq_raw
from soundsig.sound import spec_colormap

import pynwb
from matplotlib import pyplot as plt
# %matplotlib widget
plt.ion()

#### Read the data file

In [ ]:
# nwb_file = '/aquila_ssd2/lthomas/songephys_data/OperantEphys/HpiPur2667F/sites/HpiPur2667F_site02_240905_072851_pb_op/HpiPur2667F_site02_240905_072851_pb_op_ks4_lat_250215.nwb'

nwb_file = '/Users/frederictheunissen/Working Data/OperantEphys/NWB_Files/HpiPur2667F_site02_240905_072851_pb_op_ks4_lat_250215.nwb'
# Load the nwb file
preprocOptions = {} # we'll leave this empty and use default options
nwb_io =  pynwb.NWBHDF5IO(nwb_file, mode='r')
nwb = nwb_io.read()
units = nwb.units.to_dataframe()
# load the good units
good_units = units[units.group == 'good']


In [ ]:
# sample a random unit
unit = good_units.sample().iloc[0]
unit = good_units.iloc[10]       # This is a good example for a neuron with strong onset-offset response - positive onset and negative offset
# unit = good_units.iloc[7]
print("Processing unit: ", unit.name)

### Plot the stim and microphone data of a random playback trial

In [ ]:
def get_mic_data(nwb, trial):
    rate = nwb.acquisition['audio'].rate
    mic_data = nwb.acquisition['audio'].data
    start_id = int(trial.start_time * rate)
    end_id = int(trial.stop_time * rate)
    mic_trial = mic_data[start_id:end_id]
    return mic_trial[:,1], mic_trial[:,0]

In [ ]:
# lets take one trial and compare the spectrogram to the spectrogram of the microphone data
# lets get a ranodm trial
trials = nwb.intervals['playback_trials'].to_dataframe()

Repeat the cell below to get another example

In [ ]:
# Choose a ramdom trial
trial = trials.sample().iloc[0]
mic_trial, mic_copy = get_mic_data(nwb, trial)
rate = nwb.acquisition['audio'].rate

# Spectrogram paramteters.
stim_params = {}
stim_params['fband'] = 120
stim_params['nstd'] = 6
stim_params['high_freq'] = 8000
stim_params['low_freq'] = 250
stim_params['log'] = 1
stim_params['stim_rate'] = 1000  # Sample rate of spectrogram
DBNOISE = 80
# Colormap for plotting spectrograms
spec_colormap()   # defined in sound.py


# First figure for the microphone copy
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, dpi=100, figsize = (8,4))

tfrep = timefreq_raw(mic_copy, rate, 'ft', params=stim_params)
cmap = plt.get_cmap('SpectroColorMap')

minSpect = tfrep['spec'].max()-DBNOISE
maxB = tfrep['spec'].max()
ax1.imshow(tfrep['spec'], extent=[tfrep['t'][0], tfrep['t'][-1], tfrep['f'][0]*1e-3, tfrep['f'][-1]*1e-3],
                aspect='auto', interpolation='nearest', origin='lower', cmap=cmap, vmin=minSpect, vmax=maxB)
ax1.set_ylim(0, 8)
ax1.set_ylabel('Frequency (kHz)')

tval = np.arange(stop=len(mic_copy))/rate
ax2.plot(tval, mic_copy)
ax2.set_xlabel('Time (s)')

# Second copy for the microphone 
f, (ax1, ax2) = plt.subplots(2, 1, sharex=True, dpi=100, figsize = (8,4))

tfrep = timefreq_raw(mic_trial, rate, 'ft', params=stim_params)
minSpect = tfrep['spec'].max()-DBNOISE
maxB = tfrep['spec'].max()
ax1.imshow(tfrep['spec'], extent=[tfrep['t'][0], tfrep['t'][-1], tfrep['f'][0]*1e-3, tfrep['f'][-1]*1e-3],
                aspect='auto', interpolation='nearest', origin='lower', cmap=cmap, vmin=minSpect, vmax=maxB)
ax1.set_ylim(0, 8)
ax1.set_ylabel('Frequency (kHz)')

tval = np.arange(stop=len(mic_trial))/rate
ax2.plot(tval, mic_trial)
ax2.set_xlabel('Time (s)')



### Create a single trial stimulus-response data set (srData) 

In [ ]:
# SINGLE TRIAL 
srData_st = preprocSound.generate_srData_nwb_single_trials(nwb, 'playback_trials',unit.name, balanceFlg=False)
calcSegmentedModel.preprocess_srData(srData_st, plot=False, respChunkLen=respChunkLen, segmentBuffer=segmentBuffer, tdelta=0, plotFlg = False, seg_spec_lookup = nwb.processing['stimuli_spectrograms'])
print('This playback stim-response data set has %d trials.' % (len(srData_st['datasets'])))

stim_names = []
for ids, ds in enumerate(srData_st['datasets']):
    stim_names.append(ds['stim']['rawFile'])

unique_stims = np.unique(stim_names)

print('There are %d unique stimuli' % len(unique_stims))

In [ ]:
# Calculate mean onset and offet based on single trial data.
meansOnOff_st = np.zeros((2,nPoints))
nEventsOnOff_st = np.zeros((2,1))
responseAvg_st = 0
stimLenTot = 0

for ds in srData_st['datasets']:
    events = ds['events']
    nEvents = len(events['index'])
    stimLen = ds['resp']['psth_smooth'].shape[0]
    responseAvg_st += np.sum(ds['resp']['psth_smooth'])
    stimLenTot += stimLen
    for iEvent in range(nEvents):
        startInd = events['index'][iEvent]
        endInd = startInd + nPoints
        if (endInd>stimLen):
            endInd = stimLen
        if (events['onoff_feature'][iEvent][0] == 1 ):
            meansOnOff_st[0,0:endInd-startInd] = meansOnOff_st[0,0:endInd-startInd] + ds['resp']['psth_smooth'][startInd:endInd]
            nEventsOnOff_st[0] += 1
        else:
            meansOnOff_st[1,0:endInd-startInd] = meansOnOff_st[1,0:endInd-startInd] + ds['resp']['psth_smooth'][startInd:endInd]
            nEventsOnOff_st[1] += 1

meansOnOff_st[0,:] /= nEventsOnOff_st[0]
meansOnOff_st[1,:] /= nEventsOnOff_st[1]
responseAvg_st /= stimLenTot

meansOnOff_st[0,:] -= responseAvg_st
meansOnOff_st[1,:] -= responseAvg_st


In [ ]:
# Plot the average responses per-stim versus single trials.

plt.plot(meansOnOff[0,:], 'r', label='Onset PSTH')
plt.plot(meansOnOff[1,:], 'b', label='Offset PSTH')
plt.plot(meansOnOff_st[0,:], 'r--', label='Onset ST')
plt.plot(meansOnOff_st[1,:], 'b--', label='Off ST')
plt.legend()
plt.xlabel('Time (ms)')
plt.ylabel('Spike Rate (spikes/s)')